### Running Multiple Models in Parallel
This notebook demonstrates AI inference on a video file and a webcam feed using DeGirum PySDK and Hailo hardware. The pipeline is built using DeGirum Tools Streams, enabling efficient video processing with AI models.

Key Features:
- Loads and runs two object detection models:
- YOLOv8 COCO model on a traffic video.
- YOLOv8 Face model on a live webcam feed.
- Displays AI-detected objects with overlay and FPS metrics.

In [ ]:
import degirum as dg
import degirum_tools
import degirum_tools.streams as dgstreams

inference_host_address = "@local"
zoo_url = 'degirum/hailo'
token=''
device_type=['HAILORT/HAILO8L']

# Define the configurations for video file and webcam
configurations = [
    {
        "model_name": "yolov8n_relu6_coco--640x640_quant_hailort_hailo8l_1",
        "source": "../assets/Traffic.mp4",  # Video file
        "display_name": "Traffic Camera",
    },
    {
        "model_name": "yolov8n_relu6_face--640x640_quant_hailort_hailo8l_1",
        "source": 0,  # Webcam index
        "display_name": "Webcam Feed",
    },
]

# load models
models = [
    dg.load_model(
        model_name=cfg["model_name"], 
        inference_host_address=inference_host_address, 
        zoo_url=zoo_url, 
        token=token,
        device_type=device_type
    )
    for cfg in configurations
]

# define gizmos
sources = [dgstreams.VideoSourceGizmo(cfg["source"]) for cfg in configurations]
detectors = [dgstreams.AiSimpleGizmo(model) for model in models]
display = dgstreams.VideoDisplayGizmo(
    [cfg["display_name"] for cfg in configurations], show_ai_overlay=True, show_fps=True
)

# create pipeline
pipeline = (
    (source >> detector for source, detector in zip(sources, detectors)),
    (detector >> display[di] for di, detector in enumerate(detectors)),
)

# start composition
dgstreams.Composition(*pipeline).start()